# Day 16 - GPT 4o (GHC)

In [1]:
import heapq

def read_maze(filename):
    with open(filename, 'r') as file:
        maze = [list(line.strip()) for line in file]
    return maze

def find_start_end(maze):
    start = end = None
    for r, row in enumerate(maze):
        for c, val in enumerate(row):
            if val == 'S':
                start = (r, c)
            elif val == 'E':
                end = (r, c)
    return start, end

def solve_maze(maze):
    start, end = find_start_end(maze)
    directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]  # East, South, West, North
    start_dir = 0  # Initially facing East

    rows, cols = len(maze), len(maze[0])
    pq = [(0, start[0], start[1], start_dir)]  # (cost, row, col, direction)
    visited = set()

    while pq:
        cost, r, c, dir = heapq.heappop(pq)
        if (r, c, dir) in visited:
            continue
        visited.add((r, c, dir))

        if (r, c) == end:
            return cost

        # Move forward
        nr, nc = r + directions[dir][0], c + directions[dir][1]
        if 0 <= nr < rows and 0 <= nc < cols and maze[nr][nc] != '#':
            heapq.heappush(pq, (cost + 1, nr, nc, dir))

        # Turn left (counterclockwise)
        new_dir = (dir - 1) % 4
        heapq.heappush(pq, (cost + 1000, r, c, new_dir))

        # Turn right (clockwise)
        new_dir = (dir + 1) % 4
        heapq.heappush(pq, (cost + 1000, r, c, new_dir))

    return -1  # If no path is found

if __name__ == "__main__":
    maze = read_maze('input.txt')
    result = solve_maze(maze)
    print(result)

66404


## Part 2

In [2]:
import heapq

def read_maze(filename):
    with open(filename, 'r') as file:
        maze = [list(line.strip()) for line in file]
    return maze

def find_start_end(maze):
    start = end = None
    for r, row in enumerate(maze):
        for c, val in enumerate(row):
            if val == 'S':
                start = (r, c)
            elif val == 'E':
                end = (r, c)
    return start, end

def solve_maze(maze):
    start, end = find_start_end(maze)
    directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]  # East, South, West, North
    start_dir = 0  # Initially facing East

    rows, cols = len(maze), len(maze[0])
    pq = [(0, start[0], start[1], start_dir)]  # (cost, row, col, direction)
    visited = {}
    paths = {}

    while pq:
        cost, r, c, dir = heapq.heappop(pq)
        if (r, c, dir) in visited and visited[(r, c, dir)] <= cost:
            continue
        visited[(r, c, dir)] = cost

        if (r, c) == end:
            if (r, c) not in paths:
                paths[(r, c)] = []
            paths[(r, c)].append((cost, r, c, dir))
            continue

        # Move forward
        nr, nc = r + directions[dir][0], c + directions[dir][1]
        if 0 <= nr < rows and 0 <= nc < cols and maze[nr][nc] != '#':
            heapq.heappush(pq, (cost + 1, nr, nc, dir))
            if (nr, nc) not in paths:
                paths[(nr, nc)] = []
            paths[(nr, nc)].append((cost + 1, nr, nc, dir))

        # Turn left (counterclockwise)
        new_dir = (dir - 1) % 4
        heapq.heappush(pq, (cost + 1000, r, c, new_dir))
        if (r, c) not in paths:
            paths[(r, c)] = []
        paths[(r, c)].append((cost + 1000, r, c, new_dir))

        # Turn right (clockwise)
        new_dir = (dir + 1) % 4
        heapq.heappush(pq, (cost + 1000, r, c, new_dir))
        if (r, c) not in paths:
            paths[(r, c)] = []
        paths[(r, c)].append((cost + 1000, r, c, new_dir))

    return paths, end

def mark_best_paths(maze, paths, end):
    best_tiles = set()
    queue = [end]
    while queue:
        r, c = queue.pop()
        if (r, c) in best_tiles:
            continue
        best_tiles.add((r, c))
        for cost, nr, nc, dir in paths.get((r, c), []):
            if (nr, nc) not in best_tiles:
                queue.append((nr, nc))

    for r, c in best_tiles:
        if maze[r][c] == '.':
            maze[r][c] = 'O'
        elif maze[r][c] == 'S' or maze[r][c] == 'E':
            maze[r][c] = 'O'

    return maze

if __name__ == "__main__":
    maze = read_maze('input.txt')
    paths, end = solve_maze(maze)
    marked_maze = mark_best_paths(maze, paths, end)
    for row in marked_maze:
        print(''.join(row))
    print("Number of tiles part of at least one best path:", sum(row.count('O') for row in marked_maze))

#############################################################################################################################################
#...........#.........#.#.......#...........................#.............#.........#...............#.....................#................O#
#.#########.#.#######.#.#.#####.#.###.###.#####.#.#.#####.###.#.#########.#.#####.#.#########.#####.#.#####.#####.#.#####.#.###.#.#######.#.#
#.#.......#.#.#...#...#.......#.#...#.#.......#.#.#.#.....#...#.#.........#.#.....#...........#...................#.#...#.#...#...#.#.......#
#.#.###.#.#.#.###.#.###########.###.#.#######.#.#.###.#.###.###.#.#########.#.#################.#.###.###.#.#.#.#.#.#.#.#.###.#####.#.#####.#
#.......#.#.#...#.........#.....#...#...#...#...#.#...#.#...#...#...........#.....#.........#...#.#.....#.#...#.#.#.#.#.#.......#...#...#.#.#
#.#####.###.#.#.#########.#.#####.#####.#.#.###.#.#.###.#.#####.#############.###.#.#######.#.#.#.#.###.###.###.#.#.#.#.###.###.###.###.#.#.#
#.....

In [3]:
import heapq
from collections import defaultdict, deque

def read_maze(filename):
    with open(filename, 'r') as file:
        maze = [list(line.strip()) for line in file]
    return maze

def find_start_end(maze):
    start = end = None
    for r, row in enumerate(maze):
        for c, val in enumerate(row):
            if val == 'S':
                start = (r, c)
            elif val == 'E':
                end = (r, c)
    return start, end

def solve_maze(maze):
    start, end = find_start_end(maze)
    directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]  # East, South, West, North
    start_dir = 0  # Initially facing East

    rows, cols = len(maze), len(maze[0])
    pq = [(0, start[0], start[1], start_dir)]  # (cost, row, col, direction)
    visited = {}
    min_cost = float('inf')
    paths = defaultdict(list)

    while pq:
        cost, r, c, dir = heapq.heappop(pq)
        if (r, c, dir) in visited and visited[(r, c, dir)] <= cost:
            continue
        visited[(r, c, dir)] = cost

        if (r, c) == end:
            min_cost = min(min_cost, cost)
            paths[(r, c)].append((cost, r, c, dir))
            continue

        # Move forward
        nr, nc = r + directions[dir][0], c + directions[dir][1]
        if 0 <= nr < rows and 0 <= nc < cols and maze[nr][nc] != '#':
            heapq.heappush(pq, (cost + 1, nr, nc, dir))
            paths[(nr, nc)].append((cost + 1, r, c, dir))

        # Turn left (counterclockwise)
        new_dir = (dir - 1) % 4
        heapq.heappush(pq, (cost + 1000, r, c, new_dir))
        paths[(r, c)].append((cost + 1000, r, c, new_dir))

        # Turn right (clockwise)
        new_dir = (dir + 1) % 4
        heapq.heappush(pq, (cost + 1000, r, c, new_dir))
        paths[(r, c)].append((cost + 1000, r, c, new_dir))

    return paths, end, min_cost

def mark_best_paths(maze, paths, end, min_cost):
    best_tiles = set()
    queue = deque([end])
    while queue:
        r, c = queue.popleft()
        if (r, c) in best_tiles:
            continue
        best_tiles.add((r, c))
        for cost, nr, nc, dir in paths[(r, c)]:
            if cost <= min_cost and (nr, nc) not in best_tiles:
                queue.append((nr, nc))

    for r, c in best_tiles:
        if maze[r][c] == '.':
            maze[r][c] = 'O'
        elif maze[r][c] == 'S' or maze[r][c] == 'E':
            maze[r][c] = 'O'

    return maze

if __name__ == "__main__":
    maze = read_maze('input.txt')
    paths, end, min_cost = solve_maze(maze)
    marked_maze = mark_best_paths(maze, paths, end, min_cost)
    for row in marked_maze:
        print(''.join(row))
    print("Number of tiles part of at least one best path:", sum(row.count('O') for row in marked_maze))

#############################################################################################################################################
#OOOOOOOOOOO#OOOOOOOOO#O#OOOOOOO#OOOOOOOOOOOOOOOOOOOOOOOOOOO#OOOOOOOOOOOOO#OOOOOOOOO#OOOOOOOOOOOOOOO#OOOOOOOOOOOOOOOOOOOOO#OOOOOOOOOOOOOOOOO#
#O#########O#O#######O#O#O#####O#O###O###O#####O#O#O#####O###O#O#########O#O#####O#O#########O#####O#O#####O#####O#O#####O#O###.#.#######.#.#
#O#OOOOOOO#O#O#OOO#OOO#OOOOOOO#O#OOO#O#OOOOOOO#O#O#O#OOOOO#OOO#O#OOOOOOOOO#O#OOOOO#OOOOOOOOOOO#OOOOOOOOOOOOOOOOOOO#O#OOO#O#OOO#...#.#.......#
#O#O###O#O#O#O###O#O###########O###O#O#######O#O#O###O#O###O###O#O#########O#O#################O#O###O###O#O#O#O#O#O#O#O#O###O#####.#.#####.#
#OOOOOOO#O#O#OOO#OOOOOOOOO#OOOOO#OOO#OOO#OOO#OOO#O#OOO#O#OOO#OOO#OOOOOOOOOOO#OOOOO#OOOOOOOOO#OOO#O#OOOOO#O#OOO#O#O#O#O#O#OOOOOOO#...#...#.#.#
#O#####O###O#O#O#########O#O#####O#####O#O#O###O#O#O###O#O#####O#############O###O#O#######O#O#O#O#O###O###O###O#O#O#O#O###O###O###.###.#.#.#
#OOOOO

In [4]:
import heapq
from collections import defaultdict, deque

def read_maze(filename):
    with open(filename, 'r') as file:
        maze = [list(line.strip()) for line in file]
    return maze

def find_start_end(maze):
    start = end = None
    for r, row in enumerate(maze):
        for c, val in enumerate(row):
            if val == 'S':
                start = (r, c)
            elif val == 'E':
                end = (r, c)
    return start, end

def solve_maze(maze):
    start, end = find_start_end(maze)
    directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]  # East, South, West, North
    start_dir = 0  # Initially facing East

    rows, cols = len(maze), len(maze[0])
    pq = [(0, start[0], start[1], start_dir)]  # (cost, row, col, direction)
    visited = {}
    min_cost = float('inf')
    paths = defaultdict(list)

    while pq:
        cost, r, c, dir = heapq.heappop(pq)
        if (r, c, dir) in visited and visited[(r, c, dir)] <= cost:
            continue
        visited[(r, c, dir)] = cost

        if (r, c) == end:
            min_cost = min(min_cost, cost)
            paths[(r, c)].append((cost, r, c, dir))
            continue

        # Move forward
        nr, nc = r + directions[dir][0], c + directions[dir][1]
        if 0 <= nr < rows and 0 <= nc < cols and maze[nr][nc] != '#':
            heapq.heappush(pq, (cost + 1, nr, nc, dir))
            paths[(nr, nc)].append((cost + 1, r, c, dir))

        # Turn left (counterclockwise)
        new_dir = (dir - 1) % 4
        heapq.heappush(pq, (cost + 1000, r, c, new_dir))
        paths[(r, c)].append((cost + 1000, r, c, new_dir))

        # Turn right (clockwise)
        new_dir = (dir + 1) % 4
        heapq.heappush(pq, (cost + 1000, r, c, new_dir))
        paths[(r, c)].append((cost + 1000, r, c, new_dir))

    return paths, end, min_cost

def mark_best_paths(maze, paths, end, min_cost):
    best_tiles = set()
    queue = deque([end])
    while queue:
        r, c = queue.popleft()
        if (r, c) in best_tiles:
            continue
        best_tiles.add((r, c))
        for cost, nr, nc, dir in paths[(r, c)]:
            if cost <= min_cost and (nr, nc) not in best_tiles:
                queue.append((nr, nc))

    for r, c in best_tiles:
        if maze[r][c] == '.':
            maze[r][c] = 'O'
        elif maze[r][c] == 'S' or maze[r][c] == 'E':
            maze[r][c] = 'O'

    return maze

if __name__ == "__main__":
    maze = read_maze('input.txt')
    paths, end, min_cost = solve_maze(maze)
    marked_maze = mark_best_paths(maze, paths, end, min_cost)
    for row in marked_maze:
        print(''.join(row))
    print("Number of tiles part of at least one best path:", sum(row.count('O') for row in marked_maze))

#############################################################################################################################################
#OOOOOOOOOOO#OOOOOOOOO#O#OOOOOOO#OOOOOOOOOOOOOOOOOOOOOOOOOOO#OOOOOOOOOOOOO#OOOOOOOOO#OOOOOOOOOOOOOOO#OOOOOOOOOOOOOOOOOOOOO#OOOOOOOOOOOOOOOOO#
#O#########O#O#######O#O#O#####O#O###O###O#####O#O#O#####O###O#O#########O#O#####O#O#########O#####O#O#####O#####O#O#####O#O###.#.#######.#.#
#O#OOOOOOO#O#O#OOO#OOO#OOOOOOO#O#OOO#O#OOOOOOO#O#O#O#OOOOO#OOO#O#OOOOOOOOO#O#OOOOO#OOOOOOOOOOO#OOOOOOOOOOOOOOOOOOO#O#OOO#O#OOO#...#.#.......#
#O#O###O#O#O#O###O#O###########O###O#O#######O#O#O###O#O###O###O#O#########O#O#################O#O###O###O#O#O#O#O#O#O#O#O###O#####.#.#####.#
#OOOOOOO#O#O#OOO#OOOOOOOOO#OOOOO#OOO#OOO#OOO#OOO#O#OOO#O#OOO#OOO#OOOOOOOOOOO#OOOOO#OOOOOOOOO#OOO#O#OOOOO#O#OOO#O#O#O#O#O#OOOOOOO#...#...#.#.#
#O#####O###O#O#O#########O#O#####O#####O#O#O###O#O#O###O#O#####O#############O###O#O#######O#O#O#O#O###O###O###O#O#O#O#O###O###O###.###.#.#.#
#OOOOO

In [5]:
import heapq
from collections import defaultdict, deque

def read_maze(filename):
    with open(filename, 'r') as file:
        maze = [list(line.strip()) for line in file]
    return maze

def find_start_end(maze):
    start = end = None
    for r, row in enumerate(maze):
        for c, val in enumerate(row):
            if val == 'S':
                start = (r, c)
            elif val == 'E':
                end = (r, c)
    return start, end

def solve_maze(maze):
    start, end = find_start_end(maze)
    directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]  # East, South, West, North
    start_dir = 0  # Initially facing East

    rows, cols = len(maze), len(maze[0])
    pq = [(0, start[0], start[1], start_dir)]  # (cost, row, col, direction)
    visited = {}
    min_cost = float('inf')
    paths = defaultdict(list)

    while pq:
        cost, r, c, dir = heapq.heappop(pq)
        if (r, c, dir) in visited and visited[(r, c, dir)] <= cost:
            continue
        visited[(r, c, dir)] = cost

        if (r, c) == end:
            min_cost = min(min_cost, cost)
            paths[(r, c)].append((cost, r, c, dir))
            continue

        # Move forward
        nr, nc = r + directions[dir][0], c + directions[dir][1]
        if 0 <= nr < rows and 0 <= nc < cols and maze[nr][nc] != '#':
            heapq.heappush(pq, (cost + 1, nr, nc, dir))
            paths[(nr, nc)].append((cost + 1, r, c, dir))

        # Turn left (counterclockwise)
        new_dir = (dir - 1) % 4
        heapq.heappush(pq, (cost + 1000, r, c, new_dir))
        paths[(r, c)].append((cost + 1000, r, c, new_dir))

        # Turn right (clockwise)
        new_dir = (dir + 1) % 4
        heapq.heappush(pq, (cost + 1000, r, c, new_dir))
        paths[(r, c)].append((cost + 1000, r, c, new_dir))

    return paths, end, min_cost

def mark_best_paths(maze, paths, end, min_cost):
    best_tiles = set()
    queue = deque([end])
    while queue:
        r, c = queue.popleft()
        if (r, c) in best_tiles:
            continue
        best_tiles.add((r, c))
        for cost, nr, nc, dir in paths[(r, c)]:
            if cost <= min_cost and (nr, nc) not in best_tiles:
                queue.append((nr, nc))

    for r, c in best_tiles:
        if maze[r][c] == '.':
            maze[r][c] = 'O'
        elif maze[r][c] == 'S' or maze[r][c] == 'E':
            maze[r][c] = 'O'

    return maze

if __name__ == "__main__":
    maze = read_maze('input.txt')
    paths, end, min_cost = solve_maze(maze)
    marked_maze = mark_best_paths(maze, paths, end, min_cost)
    for row in marked_maze:
        print(''.join(row))
    print("Number of tiles part of at least one best path:", sum(row.count('O') for row in marked_maze))

#############################################################################################################################################
#OOOOOOOOOOO#OOOOOOOOO#O#OOOOOOO#OOOOOOOOOOOOOOOOOOOOOOOOOOO#OOOOOOOOOOOOO#OOOOOOOOO#OOOOOOOOOOOOOOO#OOOOOOOOOOOOOOOOOOOOO#OOOOOOOOOOOOOOOOO#
#O#########O#O#######O#O#O#####O#O###O###O#####O#O#O#####O###O#O#########O#O#####O#O#########O#####O#O#####O#####O#O#####O#O###.#.#######.#.#
#O#OOOOOOO#O#O#OOO#OOO#OOOOOOO#O#OOO#O#OOOOOOO#O#O#O#OOOOO#OOO#O#OOOOOOOOO#O#OOOOO#OOOOOOOOOOO#OOOOOOOOOOOOOOOOOOO#O#OOO#O#OOO#...#.#.......#
#O#O###O#O#O#O###O#O###########O###O#O#######O#O#O###O#O###O###O#O#########O#O#################O#O###O###O#O#O#O#O#O#O#O#O###O#####.#.#####.#
#OOOOOOO#O#O#OOO#OOOOOOOOO#OOOOO#OOO#OOO#OOO#OOO#O#OOO#O#OOO#OOO#OOOOOOOOOOO#OOOOO#OOOOOOOOO#OOO#O#OOOOO#O#OOO#O#O#O#O#O#OOOOOOO#...#...#.#.#
#O#####O###O#O#O#########O#O#####O#####O#O#O###O#O#O###O#O#####O#############O###O#O#######O#O#O#O#O###O###O###O#O#O#O#O###O###O###.###.#.#.#
#OOOOO

In [6]:
import heapq
from collections import defaultdict, deque

def read_maze(filename):
    with open(filename, 'r') as file:
        maze = [list(line.strip()) for line in file]
    return maze

def find_start_end(maze):
    start = end = None
    for r, row in enumerate(maze):
        for c, val in enumerate(row):
            if val == 'S':
                start = (r, c)
            elif val == 'E':
                end = (r, c)
    return start, end

def solve_maze(maze):
    start, end = find_start_end(maze)
    directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]  # East, South, West, North
    start_dir = 0  # Initially facing East

    rows, cols = len(maze), len(maze[0])
    pq = [(0, start[0], start[1], start_dir)]  # (cost, row, col, direction)
    visited = {}
    min_cost = float('inf')
    paths = defaultdict(list)

    while pq:
        cost, r, c, dir = heapq.heappop(pq)
        if (r, c, dir) in visited and visited[(r, c, dir)] <= cost:
            continue
        visited[(r, c, dir)] = cost

        if (r, c) == end:
            min_cost = min(min_cost, cost)
            paths[(r, c)].append((cost, r, c, dir))
            continue

        # Move forward
        nr, nc = r + directions[dir][0], c + directions[dir][1]
        if 0 <= nr < rows and 0 <= nc < cols and maze[nr][nc] != '#':
            heapq.heappush(pq, (cost + 1, nr, nc, dir))
            paths[(nr, nc)].append((cost + 1, r, c, dir))

        # Turn left (counterclockwise)
        new_dir = (dir - 1) % 4
        heapq.heappush(pq, (cost + 1000, r, c, new_dir))
        paths[(r, c)].append((cost + 1000, r, c, new_dir))

        # Turn right (clockwise)
        new_dir = (dir + 1) % 4
        heapq.heappush(pq, (cost + 1000, r, c, new_dir))
        paths[(r, c)].append((cost + 1000, r, c, new_dir))

    return paths, end, min_cost

def mark_best_paths(maze, paths, end, min_cost):
    best_tiles = set()
    queue = deque([end])
    while queue:
        r, c = queue.popleft()
        if (r, c) in best_tiles:
            continue
        best_tiles.add((r, c))
        for cost, nr, nc, dir in paths[(r, c)]:
            if cost <= min_cost and (nr, nc) not in best_tiles:
                queue.append((nr, nc))

    for r, c in best_tiles:
        if maze[r][c] == '.':
            maze[r][c] = 'O'
        elif maze[r][c] == 'S' or maze[r][c] == 'E':
            maze[r][c] = 'O'

    return maze

if __name__ == "__main__":
    maze = read_maze('input.txt')
    paths, end, min_cost = solve_maze(maze)
    marked_maze = mark_best_paths(maze, paths, end, min_cost)
    for row in marked_maze:
        print(''.join(row))
    print("Number of tiles part of at least one best path:", sum(row.count('O') for row in marked_maze))

#############################################################################################################################################
#OOOOOOOOOOO#OOOOOOOOO#O#OOOOOOO#OOOOOOOOOOOOOOOOOOOOOOOOOOO#OOOOOOOOOOOOO#OOOOOOOOO#OOOOOOOOOOOOOOO#OOOOOOOOOOOOOOOOOOOOO#OOOOOOOOOOOOOOOOO#
#O#########O#O#######O#O#O#####O#O###O###O#####O#O#O#####O###O#O#########O#O#####O#O#########O#####O#O#####O#####O#O#####O#O###.#.#######.#.#
#O#OOOOOOO#O#O#OOO#OOO#OOOOOOO#O#OOO#O#OOOOOOO#O#O#O#OOOOO#OOO#O#OOOOOOOOO#O#OOOOO#OOOOOOOOOOO#OOOOOOOOOOOOOOOOOOO#O#OOO#O#OOO#...#.#.......#
#O#O###O#O#O#O###O#O###########O###O#O#######O#O#O###O#O###O###O#O#########O#O#################O#O###O###O#O#O#O#O#O#O#O#O###O#####.#.#####.#
#OOOOOOO#O#O#OOO#OOOOOOOOO#OOOOO#OOO#OOO#OOO#OOO#O#OOO#O#OOO#OOO#OOOOOOOOOOO#OOOOO#OOOOOOOOO#OOO#O#OOOOO#O#OOO#O#O#O#O#O#OOOOOOO#...#...#.#.#
#O#####O###O#O#O#########O#O#####O#####O#O#O###O#O#O###O#O#####O#############O###O#O#######O#O#O#O#O###O###O###O#O#O#O#O###O###O###.###.#.#.#
#OOOOO